In [ ]:
from lusidtools.jupyter_tools import toggle_code

"""Structured Results Store for storage of Portfolio data

Attributes
----------
structured_results_store
virtual_document
luminesce
"""

toggle_code("Toggle Docstring")

# Structured Results Store Example

This notebook demostrates loading of a custom Portfolio dataset into the LUSID Structured Results Store.

For more context on what a Structure Result Store/Data is see KB Article <a ref=https://support.lusid.com/knowledgebase/article/KA-01893/en-us>KA-01893</a>

Once loaded, the dataset can be retrieved as one document, or the individual fields accessed

In [39]:
import os
import pandas as pd
from datetime import datetime, timezone
import io
import json
from IPython.core.display import HTML
from itertools import chain

# Then import the key modules from the LUSID package (i.e. The LUSID SDK)
import lusid as lu
import lusid.models as lm

# And use absolute imports to import key functions from Lusid-Python-Tools and other helper package
from lusid.utilities import ApiClientFactory

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = ApiClientFactory(
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

api_status = pd.DataFrame(
    api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict()
)

display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.9406.0,0.5.2889,"{'relation': 'RequestLogs', 'href': 'http://se..."


In [ ]:
srs_api = api_factory.build(lu.StructuredResultDataApi)

<h2>Scope, Effective Date, Result Type</h2>

<ul>
<li>Scope: The scope in which to create or update data maps.</li>
<li>Effective Date: Date at which the structured result is effective from.</li>
<li>Result Type: The Class of the Result, has to be one of [UnitResult/Analytic, UnitResult/Grouped, UnitResult/Holding]</li>
</ul>

In [ ]:
scope = "srs" 
effective_date = datetime(2021, 1, 27, tzinfo=timezone.utc)
sample_data_result_type = "UnitResult/Custom" 

## Define functions to create the Data Map


<h3>Data Mapping</h3>

The Structured Result Store allows us to upsert structured results, but to give them structure we must first define a Data Mapping,

You can think of a Data Mapping as defining the columns and the column types for each row that gets upserted into the SRS using a specific Data Map

Each "row" can have multiple keys and when difining the data map there are 4 different key types, Unique, PartOfUnique, Leaf, and CompositeLeaf

<h4>Unique</h4>
<blockquote>A primary key that will throw a unique key constraint if multiple equal values are upserted for a single specific result type</blockquote>
<h4>Part of Unique</h4>
<blockquote>A key that will be considered as part of the compoisite primary key for an entity</blockquote>
<h4>Leaf</h4>
<blockquote>Leaf define a value that wont be considered as part of the primary or primary composite key.</blockquote>
<h4>Composite Leaf</h4>
<blockquote>
CompositeLeaf is an abstraction that allows the user to specify which Leafs should be connected, they define data types rather than names of entities as they represent a group of entities.
E.g. If we have an Accural we can define a composite leaf as two leafs of amount and currency
<p></p>
<blockquote>
"UnitResult/Accrual"

DataDefinition(address="UnitResult/Accrual", dataType= "Result0D", keyType="CompositeLeaf") 
</blockquote>

<blockquote>
"UnitResult/Accrual/Amount"

DataDefinition(address="UnitResult/Accrual/Amount", name="Accrual", dataType= "decimal", keyType="Leaf")
</blockquote>
<blockquote>
"UnitResult/Accrual/Ccy"

DataDefinition(address="UnitResult/Accrual/AmountCcy", name="AccrualCcy", dataType= "string", keyType="Leaf")
</blockquote>
</blockquote>

In [ ]:
def gen_srs_data_map(leaf_columns, unique_columns, data_types):
    def define_data_definitions(columns, key_type):
        return [lm.DataDefinition(address=f"UnitResult/{scope}/{column}", name=column, data_type=data_types[column], key_type=key_type) 
                      for column in columns]
    data_map = [define_data_definitions(columns, key_type) for columns, key_type in zip((leaf_columns, unique_columns), ("Leaf", "PartOfUnique"))]

    return lm.DataMapping(data_definitions = list(chain(*data_map)))

def create_data_map(df, code, version, leaf_columns, unique_columns, data_types):
    srs_data_map = gen_srs_data_map(leaf_columns, unique_columns, data_types)
    srs_data_map_key = lm.DataMapKey(version=version, code=code)
    
    try:    
        srs_api.create_data_map(
            scope=scope, 
            request_body={
                code: lm.CreateDataMapRequest(
                    id = srs_data_map_key,
                    data = srs_data_map
                )
            }
        )
    except lu.ApiException as e:
        detail = json.loads(e.body)
        if detail['code'] not in [461]: 
            raise e
    return srs_data_map_key

## Read in sample data

In [37]:
df = pd.read_csv("data/srs_custom_data.csv")
df = df.round(0)
display(df.head(5))

,Port,Currency,Date,Sys,Value,Class
0,PortA,AUD,20210713,FSC,50000000,C
1,PortA,CAD,20210713,FSC,138234286,C
2,PortA,CHF,20210713,FSC,203402,C
3,PortA,EUR,20210713,FSC,235402950,C
4,PortA,EUR,20210715,FTC,22342054,C


## Upsert data into the Structured Results Store

In [ ]:
def insert_and_load_data(dataFrame):
    srs_ids=[]
    
    version = "1.01"
    portfolios = dataFrame.groupby("Port")
    unique_columns = ["Currency", "Date", "Sys", "Class"]
    leaf_columns = [column for column in dataFrame.columns if column not in unique_columns]
    
    sample_data_map_key = create_data_map(
        df = df,
        code = 'sample_data_map',
        version = "0.0009",
        leaf_columns = leaf_columns,
        unique_columns = unique_columns,
        data_types = { "RowId": "string", 
                       "Port": "string",
                       "Currency": "string",
                       "Date": "string",
                       "Sys": "string",
                       "Value": "decimal",
                       "Class": "string"}
    )

    for portfolio_id, pf_df in portfolios:    
        srs_id = lm.StructuredResultDataId(
            source = "Client", 
            code = portfolio_id, 
            effective_at = effective_date,
            result_type = sample_data_result_type)
    
        srs_ids.append(srs_id) 
    
        csv_data = io.StringIO()
        pf_df.to_csv(csv_data)   
        
        request_body = {
            portfolio_id: lm.UpsertStructuredResultDataRequest(
                id = srs_id,
                data = lm.StructuredResultData(
                    document_format = "csv",
                    version = version,
                    name = "Data file",
                    document = csv_data.getvalue(),
                    data_map_key = sample_data_map_key
                )
            )
        }
        
        result = srs_api.upsert_structured_result_data(
            scope=scope,
            request_body=request_body)

        display(pd.DataFrame(result.values.items()))
        
    return srs_ids


srs_ids = insert_and_load_data(df)

## Retrieve the raw data from the Structured Results Store

In [ ]:
def retrieve_data():
    portfolio_ids = df['Port'].unique()

    for portfolio_id in portfolio_ids:
        srs_id = lm.StructuredResultDataId(
            source = "Client", 
            code = portfolio_id, 
            effective_at = effective_date,
            result_type = sample_data_result_type)
        
        result = srs_api.get_structured_result_data(
            scope = scope, 
            request_body = {
                "key": srs_id
            }
        )
    
        csv_data = io.StringIO(result.values["key"].document)
        doc = pd.read_csv(csv_data)
        display(srs_id.code)
        display(doc)
        
retrieve_data()

## Extract a 'Virtual Document' from the Structured Results Store

In [42]:
def retrieve_virtual_document():
    portfolio_ids = df['Port'].unique()

    for portfolio_id in portfolio_ids:
        # Retrieve document from SRS
        srs_id = lm.StructuredResultDataId(
            source = "Client", 
            code = portfolio_id, 
            effective_at = effective_date,
            result_type = sample_data_result_type)
        
        result = srs_api.get_virtual_document(
            scope = scope, 
            request_body = {
                "key": srs_id
            }
        )
        
        # Convert to DataFrame
        result_dfs = []

        for item in result.values["key"].data:
            columns = item.row_data.columns
            values = [i.value for i in item.row_data.values]
    
            row_df = pd.DataFrame(values).T
            row_df.columns = columns
            
            for row in item.row_id.items():            
                row_df[row[0]] = [row[1]]

            result_dfs.append(row_df)
    
        all_dfs = pd.concat(result_dfs)
        
        return all_dfs
        
retrieve_virtual_document()

{'failed': {},
 'href': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://seamus-salt.lusid.com/app/insights/logs/0HMIANE4NC8HA:0000003C',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'values': {'key': {'data': [{'row_data': {'columns': ['UnitResult/srs/Value',
                                                       'UnitResult/srs/Port'],
                                           'values': [{'dimension': 0,
                                                       'result_value_type': 'ResultValueDecimal',
                                                       'value': 3535454.0},
                                                      {'result_value_type': 'ResultValueString',
                                                       'value': 'PortA'}]},
                              'row_id': {'UnitResult/srs/Class': 'C',
                       

,UnitResult/srs/Value,UnitResult/srs/Port,UnitResult/srs/Class,UnitResult/srs/Currency,UnitResult/srs/Date,UnitResult/srs/Sys
0,"3,535,454.00",PortA,C,USD,20210729,FTC
0,"203,402.00",PortA,C,CHF,20210713,FSC
0,"138,234,286.00",PortA,C,CAD,20210713,FSC
0,"16,875.00",PortA,C,USD,20210714,FTC
0,"50,000,000.00",PortA,C,AUD,20210713,FSC
0,"150,000.00",PortA,C,JPY,20210812,FTC
0,"654,659.00",PortA,C,JPY,20210713,FSC
0,"12,116.00",PortA,C,USD,20210730,FTC
0,"-174,328.00",PortA,C,USD,20210713,FSC
0,"454,654.00",PortA,C,USD,20210715,FTC


In [41]:
import lumipy as lmpy

secrets_path = os.getenv("FBN_SECRETS_PATH")

client = lmpy.get_client(api_secrets_filename=secrets_path)

client.query_and_fetch("""
@@effAt = select datetime('now');
@portfolio_data = use Lusid.UnitResult.AtomisedResult with @@effAt
  --scope=srs
  --code=PortA
  --source=Client
  --resultType=UnitResult/Custom
  --effectiveAt={@@effAt:s}
  --asAt={@@effAt:s}
  enduse;
  SELECT * FROM @portfolio_data""")


,UnitResult/srs/Class,UnitResult/srs/Currency,UnitResult/srs/Date,UnitResult/srs/Sys,DataKey,ResultValueType,ValueReal,ValueText
0,C,NZD,20210713,FSC,UnitResult/srs/Value,ResultValueDecimal,"65,454.00",NaN
1,C,NZD,20210713,FSC,UnitResult/srs/Port,ResultValueString,NaN,PortA
2,C,EUR,20210812,FTC,UnitResult/srs/Port,ResultValueString,NaN,PortA
3,C,EUR,20210812,FTC,UnitResult/srs/Value,ResultValueDecimal,"23,489,234.00",NaN
4,C,USD,20210719,FTC,UnitResult/srs/Port,ResultValueString,NaN,PortA
5,C,USD,20210719,FTC,UnitResult/srs/Value,ResultValueDecimal,"534,343.00",NaN
6,C,GBP,20210812,FTC,UnitResult/srs/Port,ResultValueString,NaN,PortA
7,C,GBP,20210812,FTC,UnitResult/srs/Value,ResultValueDecimal,"54,354,564.00",NaN
8,C,USD,20210723,FTC,UnitResult/srs/Port,ResultValueString,NaN,PortA
9,C,USD,20210723,FTC,UnitResult/srs/Value,ResultValueDecimal,"101,001.00",NaN


## Cleanup - delete the data from the Structured Results Store

In [ ]:
def delete_data():
    portfolio_ids = df['Port'].unique()

    for portfolio_id in portfolio_ids:
        # Retrieve document from SRS
        srs_id = lm.StructuredResultDataId(
            source = "Client", 
            code = portfolio_id, 
            effective_at = effective_date,
            result_type = sample_data_result_type)
        
        result = srs_api.delete_structured_result_data(
            scope = scope, 
            request_body = {
                "key": srs_id
            }
        )

delete_data()